# Bioreactor feedback control design
Based on Wikipedia pages on [control theory](https://en.wikipedia.org/wiki/Control_theory) and [PID control](https://en.wikipedia.org/wiki/PID_controller). Some quotes without attribution, do not publish as is.

**Control variable ($u(t)$)**: LED panel voltage.

**Process variable (PV)**: carbon dioxide concentration of ambient environment, measured with a Senseair S8/LP8 sensor.

**Set point (SP)**: 800 ppm.

$SP-PV$ is the error signal, which is applied as feedback to generate a control action, turning the lights up or down, in order to bring the $PV$ closer to the $SP$.

The following is a generic closed loop controller. Here, $PV-SP = e(t)$

![](https://upload.wikimedia.org/wikipedia/commons/9/90/Simple_feedback_control_loop2.svg)

The closed loop transfer function is as follows.

$$H(s)=\left(\frac{P(s)C(s)}{1+P(s)C(s)}\right)$$

where $Y(s)=H(s)R(s)$

### Finding $P(s)$
#### Determining the equation
In our case, the plant is found via system identification, assuming all variables except light intensity and photosynthesis rate are held constant. A voltage is applied to the lights, which produces a light intensity that is applied to the algae, which produces a certain photosynthesis rate. The Blackman equation for photosynthesis indicates that the photosynthesis response to light intensity will be linear, as long as the light intensity and CO2 concentration are within a range that does not saturate the photosynthesis rate. 

![](https://www.biology-pages.info/R/RateLimits.jpg)

Because photosynthesis rate is equivalent with CO2 rate, the result can then be integrated to determine the resulting CO2 concentration. The plant is represented as follows.

$$P(s)=\frac{K_L K_P}{s}$$

#### Determining the gains
From system identification experiments:

In [3]:
K_L = 0;
K_P = 0;

### Finding $C(s)$
#### Determining the equation
In order to respond to elevated levels and rates of CO2 without using too much energy (that is, without keeping the lights on high all the time), a PID controller is created. 

Term P is proportional to the current value of the $SP-PV$ error. So, if the error is large the control output, the voltage, will be proportionately large, leading to bright lights and a high photosynthesis rate.

Term I accounts for past values of the $SP−PV$ error and integrates them over time to produce the I term. For example, if there is a residual $SP−PV$ error after the application of proportional control, the integral term seeks to eliminate the residual error by adding a control effect due to the historic cumulative value of the error.

Term D is a best estimate of the future trend of the $SP−PV$ error, based on its current rate of change. The more rapid the change, the greater the controlling or damping effect. So, if the CO2 level is increasing rapidly, this term will result in a larger voltage command to the lights.

Because we want to respond to both level and cumulative effect of the CO2, let's try a PI controller. *(actually it might be a PD controller, since we want to respond to high rates with bright lights, but let's try Garbini's suggestion for now.)

The tuning constants depend on the response characteristics of the complete loop external to the controller, so we need to derive them.

In time, the PID controller will be as follows.

$$u(t)=K_pe(t)+K_i\int_0^t{e(t')dt'}+K_d\frac{de(t)}{dt}$$

Since we are building a PI controller, $K_d=0$. Taking a [Laplace transform](https://tutorial.math.lamar.edu/classes/de/laplace_table.aspx), we get

$$U(s) = K_pE(s)+K_i\frac{E(s)}{s}$$

so $$C(s) = \frac{U(s)}{E(s)} = K_p+\frac{K_i}{s}$$

#### Determining the gains

*next*
- understand fourier transform
- controller design
    - do we need lead-lag? ("Situations may occur where there are excessive delays: the measurement of the process value is delayed, or the control action does not apply quickly enough.")

### Complete feedback loop
The closed loop model is almost complete. We only need to add a disturbance in CO2 rate to represent the rate that CO2 will be added to the ambient environment by a human or another process. This results in the following closed loop system.

![]()